## Packages

In [1]:
import pandas as pd             # Traitement des tableaux de données 

# Chargement des données

In [2]:
full_data = pd.read_csv('Data/TEST/TEST/Full_Test_V2.csv', sep=',', low_memory=False)

In [3]:
full_data.head()

,Num_Acc,place,catu,sexe,trajet,an_nais,num_veh,id_vehicule_x,secu1,secu2,...,lat,long,dep,catr,circ,nbv,prof,plan,surf,situ
0,201200049538,1.0,1,0.0,1.0,1954.0,A01,NaN,1.0,NaN,...,4891660.0,235776.0,930,1,1.0,4.0,2.0,1.0,1.0,1.0
1,201200049538,1.0,1,0.0,1.0,1968.0,C01,NaN,1.0,NaN,...,4891660.0,235776.0,930,1,1.0,4.0,2.0,1.0,1.0,1.0
2,201200049538,1.0,1,0.0,2.0,1984.0,B01,NaN,1.0,NaN,...,4891660.0,235776.0,930,1,1.0,4.0,2.0,1.0,1.0,1.0
3,201200004221,1.0,1,1.0,NaN,1973.0,A01,NaN,2.0,NaN,...,4432636.0,141214.0,460,3,2.0,2.0,2.0,1.0,1.0,1.0
4,201200002457,1.0,1,1.0,NaN,1984.0,A01,NaN,1.0,NaN,...,0.0,0.0,850,3,2.0,NaN,NaN,1.0,1.0,1.0


## Traitement des données conformément à celui appliquer sur les données d'entraînement

On applique les mêmes traitements que ceux appliquer au set d'entraînement afin de conserver une homogénéité train/test assurrant au modèle de simplement travailler sur des valeurs inédites et non des formats.

### CARACTERISTIQUES

- **jour** : manquement de 0 valeur.
- **mois** : manquement de 0 valeur.
- **an** : manquement de 0 valeur. 
- **hrmn** : manquement de 0 valeur. 
- **lum** : manquement de 2 valeurs.
- **dep** : manquement de 3019 valeurs.
- **agg** : manquement de 0 valeur.
- **int** : manquement de 3719 valeurs.
- **atm** : manquement de 896 valeurs.

In [4]:
def convert_float_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    
def convert_float_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    data_frame[col_name] = data_frame[col_name].astype('category')
    
def convert_str_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype('category')
    
def convert_str_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)

### jour

In [5]:
possible_values = full_data['jour'].unique()
na_count = full_data['jour'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [12 22  3 20 14 15  7 11 30  1 23 31 21 29  9 13 25  6  5 17 27 10 28  8
 18 26 16 24 19  2  4].
Le nombre de valeurs manquantes/incorrectes est de : 0 pour un total de 142422 observations soit 0.0%.


### mois

In [6]:
possible_values = full_data['mois'].unique()
na_count = full_data['mois'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [ 8  6  2 11  5 10 12  7  3  1  9  4].
Le nombre de valeurs manquantes/incorrectes est de : 0 pour un total de 142422 observations soit 0.0%.


### an

In [7]:
# Traitement sur les années pour harmoniser la forme des étiquettes [2012 et non 12 par exemple]
full_data.loc[(full_data['an'] >= 12) & (full_data['an'] <= 18), 'an'] += 2000

possible_values = full_data['an'].unique()
na_count = full_data['an'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022].
Le nombre de valeurs manquantes/incorrectes est de : 0 pour un total de 142422 observations soit 0.0%.


#### hrmn

In [8]:
# Harmonise hr:mn de type string en int
def convert_to_int(time_str):
    if isinstance(time_str, str):
        if ':' in time_str:
            hours, minutes = map(int, time_str.split(':'))
            return hours * 100 + minutes
        else:
            return int(time_str)
    else:
        return time_str
    
full_data['hrmn'] = full_data['hrmn'].apply(convert_to_int)

# Converti en valeur numérique
convert_float_to_int(full_data, 'hrmn')

# Traitement des minutes 
def generate_intervals(start, end, step):
    intervals = []
    for i in range(start, end, step):
        intervals.append((i, i + step))
    return intervals

# Generate intervals automatically for each 100 units
intervals = generate_intervals(0, 2400, 100)
simple_values = list(range(len(intervals)))

# Function to map values to simple values based on intervals
def map_to_simple_value(value):
    for i, interval in enumerate(intervals):
        if interval[0] <= value < interval[1]:
            return simple_values[i]
    return None  # Return None if value is outside all intervals

# Apply the mapping function to the column and create a new column with the simple values
full_data['hrmn'] = full_data['hrmn'].apply(map_to_simple_value)

possible_values = full_data['hrmn'].unique()
na_count = full_data['hrmn'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [13  6 22 18 21 17 11 16 10 15 19 14 23  8  5  9  7  1 20 12  2  0  4  3].
Le nombre de valeurs manquantes/incorrectes est de : 0 pour un total de 142422 observations soit 0.0%.


### lum

In [9]:
# Harmonise les données 
full_data['lum'] = full_data['lum'].replace([-1], pd.NA).fillna(pd.NA)

possible_values = full_data['lum'].unique()
na_count = full_data['lum'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [1 2 3 5 4 <NA>].
Le nombre de valeurs manquantes/incorrectes est de : 2 pour un total de 142422 observations soit 0.0014042774290488828%.


### dep

In [10]:
# Harmonisation des départements
# Supprime les 0 superflus à gauche
full_data['dep'] = full_data['dep'].apply(lambda x: x.lstrip('0'))

# Cas des départements inexistants (100, 970...)
full_data['dep'] = full_data['dep'].apply(lambda x: x[:-1] if len(x) == 3 and x.endswith('0') else x)

# Remplace les valeurs pour la Corse
full_data['dep'] = full_data['dep'].replace({'201': '2A', '202': '2B'})

# En cas d'autres valeurs de département, supprime les lignes
# Créez une liste des codes de département valides
valid_dep_codes = list(map(str, range(1, 96))) + ['971', '972', '974', '976']

# Sélectionnez les lignes où la colonne 'dep' n'est pas dans la liste des codes de département valides
invalid_dep_rows = ~full_data['dep'].isin(valid_dep_codes)

# Assignez pd.NA à la colonne 'dep' pour les lignes où la condition est vraie
full_data.loc[invalid_dep_rows, 'dep'] = pd.NA

possible_values = full_data['dep'].unique()
na_count = full_data['dep'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : ['93' '46' '85' '45' '33' '38' '94' '34' '64' '35' '83' '49' '13' '11'
 '78' '80' '971' '30' '60' '41' '69' '44' '67' '14' '52' '59' '37' '75'
 '42' '22' '29' '31' '91' '92' '79' '95' '77' '88' '72' '18' '56' '972'
 '57' '10' '47' '17' '36' '62' '974' '73' '74' '63' '53' '76' '27' '21'
 '15' '71' '68' '66' '89' '26' '50' '90' '54' '55' '20' '65' '84' '25'
 '58' '70' '82' '40' '19' '51' <NA> '87' '23' '61' '86' '28' '24' '81'
 '39' '16' '43' '12' '48' '32' '976' '6' '9' '5' '1' '4' '2' '8' '7' '3'].
Le nombre de valeurs manquantes/incorrectes est de : 3019 pour un total de 142422 observations soit 2.1197567791492884%.


### com

In [11]:
# Suppression de la colonne com 
full_data.drop(labels=['com'], axis=1, inplace=True)

### agg

In [12]:
# Binarisation de la valeur d'agglomération
full_data['agg'] = full_data['agg'].replace({2: 0})

possible_values = full_data['agg'].unique()
na_count = full_data['agg'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [1 0].
Le nombre de valeurs manquantes/incorrectes est de : 0 pour un total de 142422 observations soit 0.0%.


### int

In [13]:
# Harmonise les données
full_data['int'] = full_data['int'].replace([-1, 0, 9], pd.NA)

possible_values = full_data['int'].unique()
na_count = full_data['int'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [1 6 3 2 <NA> 5 7 4 8].
Le nombre de valeurs manquantes/incorrectes est de : 3719 pour un total de 142422 observations soit 2.6112538793163975%.


### atm

In [14]:
# Harmonise les données
full_data['atm'] = full_data['atm'].replace([-1, 0, 9], pd.NA).fillna(pd.NA)

possible_values = full_data['atm'].unique()
na_count = full_data['atm'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [1.0 2.0 3.0 <NA> 8.0 7.0 4.0 6.0 5.0].
Le nombre de valeurs manquantes/incorrectes est de : 896 pour un total de 142422 observations soit 0.6291162882138995%.


### col

In [15]:
# Harmonise les données
full_data['col'] = full_data['col'].replace([-1], pd.NA).fillna(pd.NA)

possible_values = full_data['col'].unique()
na_count = full_data['col'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [4.0 6.0 1.0 7.0 3.0 2.0 5.0 <NA>].
Le nombre de valeurs manquantes/incorrectes est de : 384 pour un total de 142422 observations soit 0.26962126637738554%.


#### adr, lat, long, gps

In [16]:
# Suppression des colonnes adr, gps, lat, long 
full_data.drop(labels=['adr', 'gps', 'lat', 'long'], axis=1, inplace=True)

In [17]:
full_data.to_csv('Data/TEST/TEST/Full_Test_V3.csv', index=False)

In [18]:
full_data.shape

(142422, 38)